# IMPORTING LIBRARIES

In [1]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.2f}'.format
import numpy as np
from pathlib import Path
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import cm as cmaps
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
from dateutil.relativedelta import relativedelta
from collections import Counter, OrderedDict
import itertools
import re

print("Packages loaded!")

Packages loaded!


# Pipeline Controller

hk_df=load_original_data() 
hk_df=lower_case_column_names(hk_df)
##hk_df=rename_columns(hk_df)
#hk_df=drop_columns(hk_df)

hk_df=load_original_data() 

hk_df= (hk_df.pipe(lower_case_column_names)\
.pipe(rename_columns)
 #.pipe(drop_columns)

)

# READ DATA INTO DATAFRAME FROM EXISTING .CSV

In [2]:
def load_original_data():
    """
    Read production data and parse into pandas dataframe.
    """
    latest_file = r"C:\Users\p.kollhof\Documents\IRONHACK_GitHub\DA_Midterm_Project\Data\Production_Data.csv"
    df_prod_whole = pd.read_csv(latest_file, sep=";", encoding = "ISO-8859-1")
    
    return df_prod_whole

df_prod_whole = load_original_data()

# DF CLEANING

## Delete unncessary/empty colums

In [3]:
def delete_unncessary_columns(df):
    """
    Delete columns only containing NaN values or columns with effectively useless data
    """
    #try:
    cols_to_drop = []
    if any(df_prod_whole.columns.str.contains("WT1")):
        cols_to_drop_1 = [col for col in df_prod_whole.columns if "WT1" in col]
        cols_to_drop.extend(cols_to_drop_1)
    if any(df_prod_whole.columns.str.contains("WT2")):
        cols_to_drop_2 = [col for col in df_prod_whole.columns if "WT2" in col]
        cols_to_drop.extend(cols_to_drop_2)
    if any(df_prod_whole.columns.str.contains("ET2")):
        cols_to_drop_3 = [col for col in df_prod_whole.columns if "ET2" in col]
        cols_to_drop.extend(cols_to_drop_3)
    if any(df_prod_whole.columns.str.contains("FT2")):
        cols_to_drop_4 = [col for col in df_prod_whole.columns if "FT2" in col]
        cols_to_drop.extend(cols_to_drop_4)
    if any(df_prod_whole.columns.str.contains("Schneiden_Ort")):
        cols_to_drop_5 = [col for col in df_prod_whole.columns if "Schneiden_Ort" in col]
        cols_to_drop.extend(cols_to_drop_5)
    if any(df_prod_whole.columns.str.contains("Schneiden_Geraet")):
        cols_to_drop_6 = [col for col in df_prod_whole.columns if "Schneiden_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_6)
    if any(df_prod_whole.columns.str.contains("Schleifen_Ort")):
        cols_to_drop_7 = [col for col in df_prod_whole.columns if "Schleifen_Ort" in col]
        cols_to_drop.extend(cols_to_drop_7)
    if any(df_prod_whole.columns.str.contains("Schleifen_Geraet")):
        cols_to_drop_8 = [col for col in df_prod_whole.columns if "Schleifen_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_8)
    if any(df_prod_whole.columns.str.contains("NS2_Ort")):
        cols_to_drop_9 = [col for col in df_prod_whole.columns if "NS2_Ort" in col]
        cols_to_drop.extend(cols_to_drop_9)
    if any(df_prod_whole.columns.str.contains("OeO_Ort")):
        cols_to_drop_10 = [col for col in df_prod_whole.columns if "OeO_Ort" in col]
        cols_to_drop.extend(cols_to_drop_10)
    if any(df_prod_whole.columns.str.contains("OeO_Geraet")):
        cols_to_drop_11 = [col for col in df_prod_whole.columns if "OeO_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_11)
        
    else:
        raise ValueError("Columns have already been deleted!")

    cols_to_drop.extend(["Versand_Intern", "SSMA_TimeStamp", "Assembley_Teflonschlauch-Charge"])
    df.drop(cols_to_drop, axis=1, inplace=True)

    #except:
        #raise ValueError("Deleting columns was unsuccessful!")
    
    return df

df_cleaned = delete_unncessary_columns(df_prod_whole)

## Make columns lowercase and fix syntax

In [4]:
def make_columns_lowercase(df):
    
    lower_cols = [i.lower().replace(" ", "_").replace("-", "_") for i in df.columns]
    df_cleaned.columns = lower_cols    
    
    return df_cleaned

df_cleaned = make_columns_lowercase(df_cleaned)

## Remove rows with XDC-ID < `68.000`

In [5]:
def delete_rows_with_XDC_ID_pre_68k(df):
    """
    Delete all entries before XDC-ID 68.000 due to them having wrong defect IDs.
    Only applies if dataframe actually has older entries with XDC-IDs < 68.000
    """
    
    df = df[df["pdc_nummer"]>=68000]
    
    return df

#df_cleaned = delete_rows_with_XDC_ID_pre_68k(df_cleaned)

## Remove rows with non-legit XDC-ID

In [6]:
def delete_non_legit_XDC_IDs(df):
    """
    Only keep entries with legit XDC-ID.
    Anything before 44145 is not usable.
    """
    
    df = df[df["pdc_nummer"]>=44145]
    
    return df

df_cleaned = delete_non_legit_XDC_IDs(df_cleaned)

## Cast `string-dates` into `datetime-dates`

In [7]:
def date_columns_transform_into_datetime(df):
    """
    Transform all dates of string type 'YYYY-MM-DD' into dt.datetime format.
    """
    date_cols = [col for col in df.columns if "datum" in col]

    for col in date_cols:
        if df[col].dtype == "object":
            df[col] = [dt.datetime.strptime(d,'%Y-%m-%d').date() if type(d)==str 
                       else np.nan if type(d)==float
                       else d
                       for d in df[col]]
            
    return df, date_cols

df_cleaned, date_columns = date_columns_transform_into_datetime(df_cleaned)

## Remove rows without entries in selected columns

In [8]:
def delete_rows_with_nan_in_selected_cols(df, cols):
    """
    Delete rows that do not have an entry in the given columns
    """
    for col in cols:
        df[col].dropna(inplace=True)
        df.reset_index(drop=True, inplace=True)
        
    return df
    
df_cleaned = delete_rows_with_nan_in_selected_cols(df_cleaned, ["schmelzen_datum", "ns1_datum"])

## Substitute Product-`Familie` int-values with strings *("PDC", "PDC-C", "XDC")*

In [9]:
def substitute_familie_int_str(df):
    """
    Parse int values (1,2,3) into string values ("pdc", "pdc-c", "xdc")
    """
    
    if df["familie"].dtype == "int64":
        df["familie"] = ["pdc" if fam==1
                         else "pdc-c" if fam==2
                         else "xdc" 
                         for fam in df_cleaned["familie"]]
        
    return df

df_cleaned = substitute_familie_int_str(df_cleaned)

## Create `_Months` columns

In [10]:
def add_date_month_string_columns(df, date_columns):
    """
    Convert YYYY-MM-DD dt.datetime into new 'YYYY-MM' string month column. 
    """
    
    unwanted_dates = ['ts_in_datum', 'ts_out_datum', 'rt_bqc_in_datum', 'rt_bqc_out_datum']
    date_columns_wanted = [col for col in date_columns if col not in unwanted_dates]
    date_columns_months = [col.split("_")[0] + "_month" for col in date_columns_wanted]
    #date_columns_months = [col  for col in date_columns_months]
    
    #if "schmelzen_Mmnths" in df.columns:
    for idx, datum in enumerate(date_columns_wanted):
        df[date_columns_months[idx]] = [str(d)[:-3] for d in df[datum]]
    #else:
        #raise ValueError("Months columns have already been added!")
    return df

df_cleaned = add_date_month_string_columns(df_cleaned, date_columns)

## Turn string-comma-values into dot-float-values

In [26]:
def string_comma_to_dot_float(df):
    """
    Replacing commas with dots in numbers and keeping string values
    """
    cols_to_use = ['ns1_winkel_vorne','ns1_winkel_hinten','ns1_durchmesser_vorne','ns1_durchmesser_hinten',
               "ns2_center_variation", "ns2_front_angle","ns2_curve","ns2_15xlength",
               "ns2_parabolic_factor","oeo_oeo", "piezo_phase"] 
                                                            #replacing commas with dots in numbers and keeping string values
    for column in cols_to_use:
        df[column] = [float(str(x).replace(',','.')) for x in df[column]] 
     
    return df

df_cleaned = string_comma_to_dot_float(df_cleaned)

In [154]:
def clean_personal_names(df):
    """
    Func descr
    """
    
    replacements = {'GD': 'GDH',
                    'MFW': 'MFE',
                    "ME": "MFE",
                    "MF": "MFE",
                    "LAE": "AL",
                    "SF": "SFZ",
                    "DC": "AC",
                    "THANH": "TT",
                    "?": np.nan}
    
    personal_cols = [col for col in df_cleaned.columns if "personal" in col]
    
    for col in personal_cols:
        pers_list = []
        
        for op in df[col]:
            if type(op)==float or op=="?":
                pers_list.append(np.nan)
            if type(op)==str and op.isalpha():
                pers_list.append(op.split(",")[0].upper())

        pers_list = [replacements.get(word, word) for word in pers_list]

        df[col] = pers_col
        
    return df
    
df_cleaned = clean_personal_names(df_cleaned)

NameError: name 'pers_col' is not defined

In [27]:
df_cleaned

,pdc_nummer,produktion_groesse,familie,art,kommentar_produktion,grenzwert_set,defekt_arbeitsschritt_id,defekt_grund_id,schmelzen_personal,schmelzen_datum,schmelzen_ort,schmelzen_geraet,schmelzen_kapillaren_charge,ns1_personal,ns1_datum,ns1_ort,ns1_geraet,ns1_winkel_vorne,ns1_winkel_hinten,ns1_durchmesser_vorne,ns1_durchmesser_hinten,schneiden_personal,schneiden_datum,schleifen_personal,schleifen_datum,ns2_personal,ns2_datum,ns2_geraet,ns2_center_variation,ns2_front_angle,ns2_curve,ns2_15xlength,ns2_parabolic_factor,oeo_personal,oeo_datum,oeo_oeo,kleben1_personal,kleben1_datum,kleben1_ort,kleben1_geraet,kleben1_kleber_charge,kleben1_kleber_teilentnahme,kleben1_piezzo_charge,kleben2_personal,kleben2_datum,kleben2_ort,kleben2_geraet,kleben2_kleber_charge,kleben2_kleber_teilentnahme,kleben2_huelse1_charge,kleben2_isolator_charge,kleben2_huelse2_charge,assembley_personal,assembley_datum,assembley_ort,assembley_huelse3_charge,uebergabe_datum_qc,dose,bqc_personal,bqc_datum,bqc_ort,bqc_geraet,bqc_spannung,bqc_puls,bqc_volumen,bqc_auslenkung,bqc_z_offset,bqc_kommentar,ts_in_datum,ts_out_datum,rt_bqc_in_datum,rt_bqc_out_datum,uebergabe_ss,et1_datum,et1_personal,et1_spannung,et1_puls,et1_volumen,et1_auslenkung,et1_z_offset,et1_ort,et1_geraet,et1_kopfid,et1_kanal,et1_version,et1_kommentar,coatingtyp,coatingtyp_personal,ft1_typ,ft1_datum,ft1_personal,ft1_spannung,ft1_puls,ft1_volumen,ft1_auslenkung,ft1_z_offset,ft1_ort,ft1_geraet,ft1_kopfid,ft1_kanal,ft1_version,ft1_kommentar,ek_datum,ek_personal,ek_spannung,ek_puls,ek_volumen,ek_groesse,ek_auslenkung,ek_z_offset,ek_kommentar,ek_ort,ek_geraet,ek_kopfid,ek_kanal,ek_version,versand_uebergabe_datum,versand_uebergabe_personal,versand_kunde,versand_order_nr,versand_datum,versand_personal,id_aktuelles_coating,piezo_position,piezo_phase,piezo_id,ns2_linear_factor,kleben1_shore,bqc_kopfid,bqc_kanal,bqc_version,schmelzen_month,ns1_month,schneiden_month,schleifen_month,ns2_month,oeo_month,kleben1_month,kleben2_month,assembley_month,uebergabe_month,bqc_month,et1_month,ft1_month,ek_month,versand_month
0,168284,1.00,pdc,1,FH Kammer 1+3 v=68.h=58 PF hoch,1,6.00,10.00,RAP,2022-08-29,Berlin,5.00,133,CT,2022-08-30,Berlin,1.00,8.00,16.00,70.10,57.40,CT,2022-08-30,FT,2022-09-05,GT,2022-09-05,1.00,0.15,0.25,1.40,5.78,35.20,DY,2022-09-05,0.00,NaN,NaN,NaN,NaN,1.00,0.00,1.00,NaN,NaN,NaN,NaN,1.00,0.00,1.00,1.00,1.00,NaN,NaN,NaN,1.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,NaN,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,"-0,01267",NaN,NaN,NaN,NaN,2022-08,2022-08,2022-08,2022-09,2022-09,2022-09,,,,,,,,,
1,168283,1.00,pdc,1,FH Kammer 1+3 v=68.h=58 PF hoch,1,1.00,1.00,RAP,2022-08-29,Berlin,5.00,133,CT,2022-08-30,Berlin,1.00,9.00,15.00,70.00,60.20,CT,2022-08-30,FT,2022-09-05,GT,2022-09-05,1.00,0.36,0.25,1.30,5.35,33.00,DY,2022-09-05,53.35,NaN,NaN,NaN,NaN,1.00,0.00,1.00,NaN,NaN,NaN,NaN,1.00,0.00,1.00,1.00,1.00,NaN,NaN,NaN,1.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,NaN,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,"0,02313",NaN,NaN,NaN,NaN,2022-08,2022-08,2022-08,2022-09,2022-09,2022-09,,,,,,,,,
2,168282,1.00,pdc,1,FH Kammer 1+3 v=68.h=58 PF hoch,1,1.00,1.00,RAP,2022-08-29,Berlin,5.00,133,CT,2022-08-30,Berlin,1.00,17.00,31.00,69.90,59.60,CT,2022-08-30,FT,2022-09-05,GT,2022-09-05,1.00,0.43,0.68,1.10,5.46,33.40,DY,2022-09-05,53.35,NaN,NaN,NaN,NaN,1.00,0.00,1.00,NaN,NaN,NaN,NaN,1.00,0.00,1.00,1.00,1.00,NaN,NaN,NaN,1.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,NaN,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [155]:
df_cleaned["schmelzen_personal"].unique()

array(['RAP', 'TT', 'CT', 'THANH', 'Rap', 'EC', 'EF', 'AL', 'FD', 'JAB',
       'UB', 'SM', 'RP', 'PK', 'EL', 'CK', 'HO', 'NK', 'RT', 'RB', 'BP',
       'WH', 'HG', 'AK', 'FT', 'EF/CK', 'AL/WH', 'HG/EF', 'EF/AL',
       'WH/AL', 'AL/HG', 'EF/HG', 'HO/WH', 'WH/FD', 'FD/WH', 'WH/HG',
       'CK/HG', 'HD', 'FD/AL', 'AL/EF', 'WH / AL', 'CF', 'FD/HG', 'HG/AL',
       'FD/EF', 'WH/EF', 'AL/CK', 'CK/AL', 'AH', 'WH/CK', 'AL/FD',
       'CK/FD', 'CK/WH', 'WH / FD', 'HO/FD', 'Ck', 'CK / WH', 'BP/PK',
       'ChT', 'BP / PK', 'BP/ PK', 'PK/BP', 'PK/SS', 'SS', 'AL/HO',
       'CK/HO', 'HO/CK', 'SS/PK', 'El', 'HO/AL', 'PN', 'WH/PN', 'BP/EL',
       'WH/HO', 'PN/HO', 'UB/PN', 'WO', 'EL/BP', 'EL/BA', 'TM', 'PN/EF',
       'EF/PN', 'JW', 0.0, 'NH RP', 'RP/NH', 'GM', 'GM/PN/HO', 'PN/AL',
       'EF/HO', 'SO/PN', 'SO', 'So', 'PN/WH', 'EF/WH', 'WH/SO', 'EF/BS',
       'AL/PN', 'HO/PN', 'HO/EF', 'AL/ WH', 'PE/AL', 'PE', 'AL/HW'],
      dtype=object)

#### make np.nan if:
    - float
    - contains numbers
    - contains "REF"
    - len > 4

In [25]:
op_list = []

for op in df_cleaned["ft1_personal"]:
    if type(op)==str:
        op_list.append(op.split("/ | ,")[0].upper())
    if type(op)==str and "?" in op:
        op_list.append(np.nan)
        
for op in op_list:
    print(op)

SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
MB
SC
MB
SC
MB
MB
MB
MB
MB
MB
MB
MB
SC
SC
MB
MB
MS
MS
MB
SC
SC
MS
SC
SC
MS
MS
SC
SC
FH
RC
RC
MS
SC
SC
SC
SC
SC
FH
SC
SC
MS
FH
RC
FH
FH
FH
FH
FH
FH
FH
FH
FH
FH
FH
FH
FH
FH
MS
MS
MS
MS
SC
SC
SC
MFE
MS
FH
MFE
MFE
MS
MS
MS
SC
MS
SC
MFE
SC
SC
MS
MS
MS
SC
MFE
MS
SC
FH
FH
MS
SC
MB
MB
MB
MB
SC
MB
SC
MB
MB
MB
MB
MB
AL
MS
FH
RC
FH
MS
MB
MB
RC
SC
MB
MB
MB
MB
SC
MB
MB
MB
MB
MS
MS
MB
MB
FH
FH
MB
AL
RC
RC
SC
SC
SC
SC
RC
MS
SC
MB
MB
MB
MB
FH
MB
SC
AL
AL
FH
AL
MS
MB
MS
RC
MS
RC
AL
RC
SC
RC
MS
AL
AL
MB
AL
AL
RC
RC
MS
RC
RC
SC
MS
SC
RC
MB
AL
RC
MB
MS
MB
FH
MB
MB
RC
RC
RC
MB
SC
SC
SC
SC
SC
MB
MS
RC
RC
SC
RC
RC
RC
RC
RC
MS
MS
MB
FH
MB
AL
AL
AL
RC
MS
MS
MS
MS
MS
MB
MB
MB
RC
MB
RC
AL
MB
AL
FF
MB
RC
MB
SC
RC
RC
RC
RC
MB
RC
RC
RC
AL
RC
RC
MB
RC
SC
RC
MB
AL
MB
MS
MS
MS
MS
RC
MFE
SC
MS
FH
MB
RC
MB
AC
MB
MB
MB
MB
MB
MB
MB
MFE
MB
AC
MB
MB
MB
MS
SC
RC
MS
AL
MB
RC
MS
MB
MB
MB
MB
AL
AC
RC
FF
AL
MB
AL
MB
MB
MB
MB
MB
MB
RC
MB
MB
MB
MB
RC
RC
RC
FH
FH
RC
FH
RC
FH
FH
FH
MB
MB
RC
MB
MB


MB
RC
MB
RC
MB
FF
MB
MB
RC
MB
MB
RC
MB
FF
LE
RC
FF
FF
FF
LE
SC
MB
SC
RC
MB
RC
RC
LE
SC
SC
MB
MB
MB
LE
MB
FF
MB
FF
MB
LE
MB
MB
LE
SC
MB
MB
SC
SC
MB
RC
SC
LE
FF
FF
MB
MB
RC
FF
MB
MB
AT
RC
LE
MB
MB
MB
MB
MB
MB
MB
FH
LE
FF
FF
FF
FF
FF
MB
MB
MB
LE
FF
FF
MS
MB
MS
SC
LE
MB
RC
MS
RC
MB
MS
LE
MB
MB
MB
RC
RC
LE
MB
MS
FF
LE
FF
FF
FF
LE
AT
LE
MB
MB
LE
LE
LE
LE
MB
MS
FF
LE
MB
SC
FF
MB
RC
MB
SC
LE
MB
MB
MB
MB
MS
MB
LE
LE
MB
RC
RC
RC
FF
LE
MB
MB
RC
SC
MB
RC
FF
MB
LE
SC
MB
MB
MB
RC
RC
FF
RC
RC
LE
MB
RC
LE
MB
RC
MB
MB
RC
MB
RC
LE
RC
RC
RC
RC
RC
RC
AT
MS
RC
MS
RC
FF
RC
RC
LE
RC
RC
LE
FF
RC
LE
LE
RC
RC
RC
RC
MB
MB
MB
MB
MB
LE
LE
MB
SC
MB
RC
RC
LE
LE
MB
MB
RC
MB
LE
MB
LE
FF
LE
FF
LE
FF
FF
LE
FF
LE
LE
FF
FF
MB
RC
LE
SC
SC
MS
LE
MB
SC
FH
FF
FH
MB
RC
MB
RC
AT
MB
MB
MB
RC
MB
MB
MB
MB
RC
SC
MB
MB
SC
LE
FF
MB
MB
MB
FF
LE
RC
SC
MB
RC
MB
LE
MB
MB
MS
MB
FF
MB
MB
MB
MB
RC
RC
FH
MB
AL
MFE
MB
MB
RC
MB
SC
MB
MB
FF
MB
MB
MB
MB
LE
SC
LE
MB
RC
FF
MB
MB
SC
MB
MB
SC
MB
MB
LE
MB
AT
SC
MB
RC
MB
LE
RC
RC
LE
MB
LE
LE
SC
MB
SC


LE
LE
LE
LE
LE
LE
FF
LE
FF
RC
SC
SC
MB
AC
AC
LE
LE
LE
LE
AC
AC
LE
LE
LE
SC
SC
SC
SC
FF
LE
LE
LE
MB
SC
LE
FF
SC
FF
AC
RC
SC
LE
FF
LE
SC
LE
SC
FF
SC
FF
LE
LE
SC
LE
SC
SC
SC
AC
SC
AC
LE
LE
AC
FF
SC
SC
SC
SC
AC
FF
MB
FF
AC
AC
LE
SC
LE
LE
LE
LE
LE
LE
SC
LE
FF
FF
AT
LE
LE
FF
LE
FF
LE
FF
LE
SC
SC
FF
RC
FF
SC
LE
LE
SC
SC
FF
LE
SC
SC
SC
FF
SC
FF
SC
SC
SC
SC
SC
AC
FF
LE
LE
LE
FF
LE
AC
FF
FF
LE
SC
LE
SC
LE
SC
RC
SC
RC
SC
SC
SC
AC
SC
SC
SC
MS
SC
SC
SC
FF
SC
AC
FF
SC
FF
SC
FF
SC
AC
AC
AC
SC
SC
SC
SC
AC
SC
SC
SC
RC
AC
FF
AC
SC
SC
LE
SC
FF
LE
SC
AC
AC
FF
SC
SC
FF
SC
FF
AC
LE
SC
SC
FF
SC
SC
AC
FF
SC
LE
SC
MB
AC
SC
SC
SC
AC
FF
SC
MB
FF
MB
SC
SC
LE
SC
SC
SC
MB
SC
SC
AC
AC
AC
SC
FF
LE
FF
SC
FF
FF
FF
DC
SC
SC
SC
SC
SC
SC
SC
SC
LE
FF
FF
LE
AC
SC
FF
FF
FF
FH
FF
SC
FF
SC
LE
AC
FF
FF
MS
RC
FF
SC
MS
FF
FF
MS
FF
AC
MB
MB
FF
LE
LE
SC
AC
LE
SC
SC
FF
SC
RC
SC
FF
SC
SC
MB
AC
AC
SC
SC
LE
SC
SC
LE
AC
FF
SC
LE
LE
SC
SC
FF
LE
SC
SC
SC
FF
LE
LE
LE
LE
FF
SC
SC
AC
SC
SC
SC
FF
SC
SC
AC
SC
SC
FF
FF
LE
AC
RC
AC
SC
SC
LE
LE
L

FF
FF
FF
MFE
FF
FF
FF
FF
FF
FF
FF
FF
AC
MFE
FF
FF
MFE
FF
FF
FF
FF
FF
FF
MFE
FF
FF
FF
FF
FF
FF
MFE
FF
MFE
MFE
FF
MFE
FF
AC
MFE
FF
FF
FF
FF
FF
MFE
FF
AC
AC
FF
AC
FF
AC
AC
AC
FF
AC
AC
FF
AC
AC
AC
AC
FF
AC
AC
FF
MFE
MFE
FF
FF
FF
FF
FF
AC
FF
FF
FF
AC
AC
AC
FF
AC
FF
AC
AC
AC
AC
AC
MFE
AC
AC
FF
AC
AC
AC
AC
FF
AC
AC
AC
FF
AC
AC
AC
AC
AC
MFE
AC
AC
AC
AC
AC
MFE
AC
AC
FF
AC
AC
AC
AC
AC
MFE
AC
AC
AC
AC
AC
AC
MFE
AC
AC
AC
AC
LE
FF
FF
FF
AC
AC
FF
AC
AC
AC
AC
FF
MFE
AC
LE
AC
MFE
AC
AC
AC
AC
AC
AC
AC
AC
AC
AC
AC
AC
AC
AC
AC
AC
MFE
AC
AC
AC
AC
AC
AC
AC
AC
AC
FF
AC
MFE
MFE
AC
AC
AC
AC
AC
AC
AC
AC
AC
FF
AC
AC
FF
MFE
AC
AC
AC
AC
AC
MFE
AC
AC
FF
MFE
MFE
MFE
AC
AC
AC
MFE
FF
AC
AC
AC
MFE
MFE
AC
AC
AC
AC
AC
AC
FF
AC
AC
MFE
AC
MFE
FF
LE
FF
FF
MFE
FF
MFE
FF
FF
FF
FF
WB
FF
FF
MFE
FF
FF
FF
MFE
FF
FF
FF
FF
AC
FF
FF
FF
FF
FF
FF
FF
FF
FF
FF
FF
FF
MFE
FF
FF
FF
FF
WB
FF
FF
FF
FF
FF
FF
MFE
MFE
FF
MFE
FF
FF
FF
FF
FF
FF
AC
FF
MFE
AC
BR
LE
FF
FF
FF
FF
MFE
AC
FF
MFE
FF
FF
MFE
FF
MFE
MFE
FF
FF
FF
FF
MFE
MFE
BR
MFE
MFE
MFE
B

In [162]:
test = []
for op in df_cleaned["schmelzen_personal"]:
    #print(op)
    if type(op)==str:
        token = re.split("[ :;'?=()!\\[\\]-]+|(?<=\\d)(?=\\D)", op)
        #token = token.split("/")[0]
    else:
        token = np.nan
    test.append(token)
    #tokens = [token for token in tokens if len(token) != 0] # remove empty strings!

In [134]:
personal_cols = [col for col in df_cleaned.columns if "personal" in col]
for col in personal_cols:
    print(col, len(df_cleaned[col]))
    print(set(df_cleaned[col]))
    print("\n")

schmelzen_personal 112972
{0.0, 'CK/HG', 'BP/ PK', 'JW', 'SO', 'WH/HO', 'PN/EF', 'HO/AL', 'PK/SS', 'AL/HG', 'RP/NH', 'EF', 'WH/FD', 'AK', 'Ck', 'GM', 'PE', 'HO', 'El', 'WH', 'RP', 'AL/EF', 'WH/CK', 'HG', 'PE/AL', 'AL/FD', 'GM/PN/HO', 'CF', 'SS', 'UB', 'FD/AL', 'CK/AL', 'AL/ WH', 'WH/AL', 'AL/WH', 'BP/EL', 'THANH', 'FD/HG', 'HG/EF', 'Rap', 'PN/HO', 'EL', 'HO/WH', 'FD/EF', 'ChT', 'So', 'EL/BA', 'TT', 'BP', 'BP/PK', 'HO/EF', 'EL/BP', 'RT', 'WH/HG', 'WH / FD', 'CT', 'EF/WH', 'CK', 'PK/BP', 'EF/PN', 'FT', 'WO', 'EF/BS', 'NK', 'AL/HW', 'PK', 'EC', 'RB', 'TM', 'RAP', 'FD', 'WH/EF', 'CK/FD', 'HD', 'EF/AL', 'HO/PN', 'EF/HO', 'JAB', 'FD/WH', 'CK/WH', 'WH / AL', 'PN/AL', 'SO/PN', 'NH RP', 'PN/WH', 'EF/CK', 'CK / WH', 'AL/PN', 'HO/CK', 'WH/SO', 'AH', 'BP / PK', 'AL/CK', 'PN', 'AL', 'HO/FD', 'CK/HO', 'EF/HG', 'AL/HO', 'WH/PN', 'UB/PN', 'HG/AL', 'SM', 'SS/PK'}


ns1_personal 112972
{'HO/GM', 0.0, 'FD/AL', 'UB', 'AK/RP', 'CT/EL', 'PN/DM', 'RP/AL', 'FT', 'HO/AL', 'EL/RP', 'EL/TM', 'DM/AL', 'EF', 'NK',

In [ ]:
personal_cols = [col for col in df_cleaned.columns if "personal" in col]
for col in personal_cols:
    pers_col = []

    for op in df[col]:
        if type(op)==float or op=="?":
            pers_col.append(np.nan)
        if type(op)==str and op.isalpha():
            pers_col.append(op.split(",")[0].upper())

    pers_col = [replacements.get(word, word) for word in pers_col]

    df[col] = pers_col

In [64]:
df_cleaned[df_cleaned["ft1_personal"]=="?"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 8714 to 48464
Columns: 146 entries, pdc_nummer to versand_month
dtypes: float64(47), int64(4), object(95)
memory usage: 54.0+ KB


In [32]:
#df_cleaned['NS1_Winkel_vorne']=pd.to_numeric(df_cleaned['NS1_Winkel_vorne'])
#df_cleaned['NS1_Winkel_hinten']=pd.to_numeric(df_cleaned['NS1_Winkel_hinten'])

#df_cleaned['NS2_Center_Variation']=pd.to_numeric(df_cleaned['NS2_Center_Variation'])
#df_cleaned['NS2_Front_Angle']=pd.to_numeric(df_cleaned['NS2_Front_Angle'])
#df_cleaned['NS2_Curve']=pd.to_numeric(df_cleaned['NS2_Curve'])
#df_cleaned['NS2_15xLength']=pd.to_numeric(df_cleaned['NS2_15xLength'])
#df_cleaned['NS2_Parabolic_Factor']=pd.to_numeric(df_cleaned['NS2_Parabolic_Factor'])

#df_cleaned['OeO_OeO']=pd.to_numeric(df_cleaned['OeO_OeO'])

ValueError: Unable to parse string "8,8" at position 423

In [2]:


##############################################
### FOLDER & PATH CREATION FOR DATA SAVING ###

### CREATE FOLDERS
Subfolder_Names = ["Scraps", "Bonding_Assembly", 
                   "OP_ThruPut_General", "OP_ThruPut_Melting", "MD_ThruPut_Melting", "FH"]
try:
    os.makedirs('Data_created_' + dt.datetime.now().strftime("%y-%m-%d") + "_PROD" + "\\")
    for subfolder in Subfolder_Names:
        os.makedirs('Data_created_' + dt.datetime.now().strftime("%y-%m-%d") + "_PROD" + "\\" + subfolder)
except FileExistsError:
    pass

### CREATE PATH
savePath = os.getcwd() + "\\" + 'Data_created_' + dt.datetime.now().strftime("%y-%m-%d") + "_PROD" + "\\"
savePath_Scrap = os.getcwd() + "\\" + 'Data_created_' + dt.datetime.now().strftime("%y-%m-%d") + "_PROD" + "\\" + "Scraps" + "\\"
savePath_BondAss = os.getcwd() + "\\" + 'Data_created_' + dt.datetime.now().strftime("%y-%m-%d") + "_PROD" + "\\" + "Bonding_Assembly" + "\\"
savePath_OP_TP_General = os.getcwd() + "\\" + 'Data_created_' + dt.datetime.now().strftime("%y-%m-%d") + "_PROD" + "\\" + "OP_ThruPut_General" + "\\"
savePath_OP_TP_Melt = os.getcwd() + "\\" + 'Data_created_' + dt.datetime.now().strftime("%y-%m-%d") + "_PROD" + "\\" + "OP_ThruPut_Melting" + "\\"
savePath_MD_TP_Melt = os.getcwd() + "\\" + 'Data_created_' + dt.datetime.now().strftime("%y-%m-%d") + "_PROD" + "\\" + "MD_ThruPut_Melting" + "\\"

savePath_FH = os.getcwd() + "\\" + 'Data_created_' + dt.datetime.now().strftime("%y-%m-%d") + "_PROD" + "\\" + "FH5" + "\\"

### FOLDER & PATH CREATION FOR DATA SAVING ###
##############################################


### COLOR SCHEMES FOR PLOTS
cmap_PDC_Sizes = ["grey", "gold", "orange", "red", "green", "mediumblue", "darkviolet"]
cmap_Coating_Types = ["grey", "gold", "red", "green", "mediumblue"]
cmap_fQC_Types = ["grey", "silver", "red", "green", "mediumblue"]
cmap_QC_Types = ["grey", "red", "green", "mediumblue"]
cmap_OPs = ["orangered", "forestgreen", "deepskyblue", "orange", "cyan", "fuchsia", "dimgrey"]

###########################################
### PROCESS STEP DATA HANDLING - MONTHS ###


### UPDATE DATAFRAME OPERATORS WITH CAPITALIZATION & SINGLE-OP ###
Schmelzen_OP_List = list(df_DB_trunc["Schmelzen_Personal"])
Schneiden_OP_List = list(df_DB_trunc["Schneiden_Personal"])
NS1_OP_List = list(df_DB_trunc["NS1_Personal"])
Schleifen_OP_List = list(df_DB_trunc["Schleifen_Personal"])
NS2_OP_List = list(df_DB_trunc["NS2_Personal"])
OeO_OP_List = list(df_DB_trunc["OeO_Personal"])
K1_OP_List = list(df_DB_trunc["Kleben1_Personal"])
K2_OP_List = list(df_DB_trunc["Kleben2_Personal"])
Ass_OP_List = list(df_DB_trunc["Assembley_Personal"])
BQC_OP_List = list(df_DB_trunc["BQC_Personal"])
ET_OP_List = list(df_DB_trunc["ET1_Personal"])
FT_OP_List = list(df_DB_trunc["FT1_Personal"])
EK_OP_List = list(df_DB_trunc["EK_Personal"])

for schmop in range(len(Schmelzen_OP_List)):
    if Schmelzen_OP_List[schmop] == 0 or type(Schmelzen_OP_List[schmop])==float:
        pass
    elif Schmelzen_OP_List[schmop].isupper():
        Schmelzen_OP_List[schmop] = Schmelzen_OP_List[schmop][:2]
    elif not Schmelzen_OP_List[schmop].isupper():
        Schmelzen_OP_List[schmop] = Schmelzen_OP_List[schmop][:2].upper()

for schnop in range(len(Schneiden_OP_List)):
    if Schneiden_OP_List[schnop] == 0 or type(Schneiden_OP_List[schnop])==float:
        pass
    elif Schneiden_OP_List[schnop].isupper():
        Schneiden_OP_List[schnop] = Schneiden_OP_List[schnop][:2]
    elif not Schneiden_OP_List[schnop].isupper():
        Schneiden_OP_List[schnop] = Schneiden_OP_List[schnop][:2].upper()
        
for ns1op in range(len(NS1_OP_List)):
    if NS1_OP_List[ns1op] == 0 or type(NS1_OP_List[ns1op])==float:
        pass
    elif NS1_OP_List[ns1op].isupper():
        NS1_OP_List[ns1op] = NS1_OP_List[ns1op][:2]
    elif not NS1_OP_List[ns1op].isupper():
        NS1_OP_List[ns1op] = NS1_OP_List[ns1op][:2].upper()

for ns2op in range(len(NS2_OP_List)):
    if NS2_OP_List[ns2op] == 0 or type(NS2_OP_List[ns2op])==float:
        pass
    elif NS2_OP_List[ns2op].isupper():
        NS2_OP_List[ns2op] = NS2_OP_List[ns2op][:2]
    elif not NS2_OP_List[ns2op].isupper():
        NS2_OP_List[ns2op] = NS2_OP_List[ns2op][:2].upper()

for oeoop in range(len(OeO_OP_List)):
    if OeO_OP_List[oeoop] == 0 or type(OeO_OP_List[oeoop])==float:
        pass
    elif OeO_OP_List[oeoop].isupper():
        OeO_OP_List[oeoop] = OeO_OP_List[oeoop][:2]
    elif not OeO_OP_List[oeoop].isupper():
        OeO_OP_List[oeoop] = OeO_OP_List[oeoop][:2].upper()

for k1op in range(len(K1_OP_List)):
    if K1_OP_List[k1op] == 0 or type(K1_OP_List[k1op])==float:
        pass
    elif K1_OP_List[k1op].isupper():
        K1_OP_List[k1op] = K1_OP_List[k1op][:2]
    elif not K1_OP_List[k1op].isupper():
        K1_OP_List[k1op] = K1_OP_List[k1op][:2].upper()
        
for k2op in range(len(K2_OP_List)):
    if K2_OP_List[k2op] == 0 or type(K2_OP_List[k2op])==float:
        pass
    elif K2_OP_List[k2op].isupper():
        K2_OP_List[k2op] = K2_OP_List[k2op][:2]
    elif not K2_OP_List[k2op].isupper():
        K2_OP_List[k2op] = K2_OP_List[k2op][:2].upper()
        
for assop in range(len(Ass_OP_List)):
    if Ass_OP_List[assop] == 0 or type(Ass_OP_List[assop])==float:
        pass
    elif Ass_OP_List[assop].isupper():
        Ass_OP_List[assop] = Ass_OP_List[assop][:2]
    elif not Ass_OP_List[assop].isupper():
        Ass_OP_List[assop] = Ass_OP_List[assop][:2].upper()
        
for bqcop in range(len(BQC_OP_List)):
    if BQC_OP_List[bqcop] == 0 or type(BQC_OP_List[bqcop])==float:
        pass
    elif BQC_OP_List[bqcop].isupper():
        BQC_OP_List[bqcop] = BQC_OP_List[bqcop][:3]
    elif not BQC_OP_List[bqcop].isupper():
        BQC_OP_List[bqcop] = BQC_OP_List[bqcop][:3].upper()
        
for etop in range(len(ET_OP_List)):
    if ET_OP_List[etop] == "MF":
        ET_OP_List[etop]=="MFE"
    if ET_OP_List[etop] == 0 or type(ET_OP_List[etop])==float:
        pass
    elif ET_OP_List[etop].isupper():
        ET_OP_List[etop] = ET_OP_List[etop][:3]
    elif not ET_OP_List[etop].isupper():
        ET_OP_List[etop] = ET_OP_List[etop][:3].upper()
    if type(ET_OP_List[etop])!=float:
        ET_OP_List[etop] = re.sub(r'[^\w]', '', ET_OP_List[etop])
        ET_OP_List[etop] = ''.join(i for i in ET_OP_List[etop] if not i.isdigit())

FT_OP_List_ = []
for ftop in range(len(FT_OP_List)):
    if FT_OP_List[ftop] == "MF":
        FT_OP_List_.append("MFE")
    elif FT_OP_List[ftop] == "SC_":
        FT_OP_List_.append("SC")
    elif FT_OP_List[ftop] == 0 or type(FT_OP_List[ftop])==float:
        FT_OP_List_.append(np.nan)
    elif FT_OP_List[ftop].isupper():
        FT_OP_List_.append(FT_OP_List[ftop][:3])
    elif not FT_OP_List[ftop].isupper():
        FT_OP_List_.append(FT_OP_List[ftop][:3].upper())
    elif type(FT_OP_List[ftop])!=float:
        FT_OP_List_.append(re.sub(r'[^\w]', '', FT_OP_List[ftop]))
        FT_OP_List_.append(''.join(i for i in FT_OP_List[ftop] if not i.isdigit()))
        
for ekop in range(len(EK_OP_List)):
    if EK_OP_List[ekop] == "MF":
        EK_OP_List[ekop]=="MFE"
    if EK_OP_List[ekop] == 0 or type(EK_OP_List[ekop])==float:
        pass
    elif EK_OP_List[ekop].isupper():
        EK_OP_List[ekop] = EK_OP_List[ekop][:3]
    elif not EK_OP_List[ekop].isupper():
        EK_OP_List[ekop] = EK_OP_List[ekop][:3].upper()
    if type(EK_OP_List[ekop])!=float:
        EK_OP_List[ekop] = re.sub(r'[^\w]', '', EK_OP_List[ekop])
        EK_OP_List[ekop] = ''.join(i for i in EK_OP_List[ekop] if not i.isdigit())

df_DB_trunc["Schmelzen_Personal"] = Schmelzen_OP_List
df_DB_trunc["Schneiden_Personal"] = Schneiden_OP_List
df_DB_trunc["NS1_Personal"] = NS1_OP_List
df_DB_trunc["NS2_Personal"] = NS2_OP_List
df_DB_trunc["OeO_Personal"] = OeO_OP_List
df_DB_trunc["Kleben1_Personal"] = K1_OP_List
df_DB_trunc["Kleben2_Personal"] = K2_OP_List
df_DB_trunc["Assembley_Personal"] = Ass_OP_List
df_DB_trunc["BQC_Personal"] = BQC_OP_List
df_DB_trunc["ET1_Personal"] = ET_OP_List
df_DB_trunc["FT1_Personal"] = FT_OP_List
df_DB_trunc["EK_Personal"] = EK_OP_List

### CREATE LISTS OF UNIQUE OPs FOR EACH PROCESS STEP ###
Schmelzen_OP_List_Set = sorted([schmop for schmop in list(set(Schmelzen_OP_List)) if type(schmop)==str if len(schmop) > 1 if len(schmop) <= 3 if "#" not in schmop if "." not in schmop])
Schneiden_OP_List_Set = sorted([schnop for schnop in list(set(Schneiden_OP_List)) if type(schnop)==str if len(schnop) > 1 if len(schnop) <= 3 if "#" not in schnop if "." not in schnop])
NS1_OP_List_Set = sorted([ns1op for ns1op in list(set(NS1_OP_List)) if type(ns1op)==str if len(ns1op) > 1 if len(ns1op) <= 3 if "#" not in ns1op if "." not in ns1op])
Schleifen_OP_List_Set = sorted([schlop for schlop in list(set(Schleifen_OP_List)) if type(schlop)==str if len(schlop) > 1 if len(schlop) <= 3 if "#" not in schlop if "." not in schlop])
NS2_OP_List_Set = sorted([ns2op for ns2op in list(set(NS2_OP_List)) if type(ns2op)==str if len(ns2op) > 1 if len(ns2op) <= 3 if "#" not in ns2op if "." not in ns2op])
OeO_OP_List_Set = sorted([oeoop for oeoop in list(set(OeO_OP_List)) if type(oeoop)==str if len(oeoop) > 1 if len(oeoop) <= 3 if "#" not in oeoop if "." not in oeoop])
K1_OP_List_Set = sorted([k1op for k1op in list(set(K1_OP_List)) if type(k1op)==str if len(k1op) > 1 if len(k1op) <= 3 if "#" not in k1op if "." not in k1op])
K2_OP_List_Set = sorted([k2op for k2op in list(set(K2_OP_List)) if type(k2op)==str if len(k2op) > 1 if len(k2op) <= 3 if "#" not in k2op if "." not in k2op])
Ass_OP_List_Set = sorted([assop for assop in list(set(Ass_OP_List)) if type(assop)==str if len(assop) > 1 if len(assop) <= 3 if "#" not in assop if "." not in assop])
BQC_OP_List_Set = sorted([bqcop for bqcop in list(set(BQC_OP_List)) if type(bqcop)==str if len(bqcop) > 1 if "#" not in bqcop if "." not in bqcop])
ET_OP_List_Set = sorted([etop for etop in list(set(ET_OP_List)) if type(etop)==str if len(etop) > 1 if "#" not in etop if "." not in etop])
FT_OP_List_Set = sorted([ftop for ftop in list(set(FT_OP_List_)) if type(ftop)==str if len(ftop) > 1 if "#" not in ftop if "." not in ftop if "_" not in ftop if "?" not in ftop])
EK_OP_List_Set = sorted([ekop for ekop in list(set(EK_OP_List)) if type(ekop)==str if len(ekop) > 1 if "#" not in ekop if "." not in ekop])

### CREATE LIST OF OVERALL UNIQUE OPERATORS
OPs_All_List = [Schmelzen_OP_List_Set, Schneiden_OP_List_Set, NS1_OP_List_Set, 
                Schleifen_OP_List_Set, NS2_OP_List_Set, OeO_OP_List_Set, K1_OP_List_Set, 
                K2_OP_List_Set, Ass_OP_List_Set, BQC_OP_List_Set]
OPs_All_List = sorted(list(set(list(itertools.chain(*OPs_All_List)))))



### TRANSFORM STRING-COMMA-VALUES TO DOT-FLOAT-VALUES ###
NS1VV_StrCommaVal_List = list(df_DB_trunc["NS1_Durchmesser_vorne"])
NS1VV_FloatDotVal_List = []
for ns1vv in NS1VV_StrCommaVal_List:
    if type(ns1vv) == float:
        NS1VV_FloatDotVal_List.append(0)
    elif type(ns1vv) == str:
        ns1vv = ns1vv.replace(',', '.')
        NS1VV_FloatDotVal_List.append(float(ns1vv))
df_DB_trunc["NS1_Durchmesser_vorne"] = NS1VV_FloatDotVal_List

NS1VH_StrCommaVal_List = list(df_DB_trunc["NS1_Durchmesser_hinten"])
NS1VH_FloatDotVal_List = []
for ns1vh in NS1VH_StrCommaVal_List:
    if type(ns1vh) == float:
        NS1VH_FloatDotVal_List.append(0)
    elif type(ns1vh) == str:
        ns1vh = ns1vh.replace(',', '.')
        NS1VH_FloatDotVal_List.append(float(ns1vh))
df_DB_trunc["NS1_Durchmesser_hinten"] = NS1VH_FloatDotVal_List

NS2PF_StrCommaVal_List = list(df_DB_trunc["NS2_Parabolic_Factor"])
NS2PF_FloatDotVal_List = []
for ns2pf in NS2PF_StrCommaVal_List:
    if type(ns2pf) == float:
        NS2PF_FloatDotVal_List.append(0)
    elif type(ns2pf) == str:
        ns2pf = ns2pf.replace(',', '.')
        NS2PF_FloatDotVal_List.append(float(ns2pf))
df_DB_trunc["NS2_Parabolic_Factor"] = NS2PF_FloatDotVal_List

NS215x_StrCommaVal_List = list(df_DB_trunc["NS2_15xLength"])
NS215x_FloatDotVal_List = []
for ns215x in NS215x_StrCommaVal_List:
    if type(ns215x) == float:
        NS215x_FloatDotVal_List.append(0)
    elif type(ns215x) == str:
        ns215x = ns215x.replace(',', '.')
        NS215x_FloatDotVal_List.append(float(ns215x))
df_DB_trunc["NS2_15xLength"] = NS215x_FloatDotVal_List

NS2CV_StrCommaVal_List = list(df_DB_trunc["NS2_Center_Variation"])
NS2CV_FloatDotVal_List = []
for ns2cv in NS2CV_StrCommaVal_List:
    if type(ns2cv) == float:
        NS2CV_FloatDotVal_List.append(0)
    elif type(ns2cv) == str:
        ns2cv = ns2cv.replace(',', '.')
        NS2CV_FloatDotVal_List.append(float(ns2cv))
df_DB_trunc["NS2_Center_Variation"] = NS2CV_FloatDotVal_List

NS2FA_StrCommaVal_List = list(df_DB_trunc["NS2_Front_Angle"])
NS2FA_FloatDotVal_List = []
for ns2fa in NS2FA_StrCommaVal_List:
    if type(ns2fa) == float:
        NS2FA_FloatDotVal_List.append(0)
    elif type(ns2fa) == str:
        ns2fa = ns2fa.replace(',', '.')
        NS2FA_FloatDotVal_List.append(float(ns2fa))
df_DB_trunc["NS2_Front_Angle"] = NS2FA_FloatDotVal_List

OeO_StrCommaVal_List = list(df_DB_trunc["OeO_OeO"])
OeO_FloatDotVal_List = []
for oeo in OeO_StrCommaVal_List:
    if type(oeo) == float:
        OeO_FloatDotVal_List.append(0)
    elif type(oeo) == str:
        oeo = oeo.replace(',', '.')
        OeO_FloatDotVal_List.append(float(oeo))
df_DB_trunc["OeO_OeO"] = OeO_FloatDotVal_List

### SORT GENERAL DATA BY PRODUCTION SITE ###
df_DB_Dortmund = df_DB_trunc[df_DB_trunc["Schmelzen_Ort"]=="Dortmund"].reset_index(drop=True)
df_DB_Berlin = df_DB_trunc[df_DB_trunc["Schmelzen_Ort"]=="Berlin"].reset_index(drop=True)

print("Dataframe loaded!")

Dataframe loaded!


In [3]:
### DETERMINE TIMEFRAME TO PLOT ###
LastNMonths = []
TimeFrame = int(input("die letzten X Monate: "))
TimeFrame = list(range(0,TimeFrame,1))
for l in TimeFrame:
    last_month = dt.datetime.now() - relativedelta(months=l)
    last_month_ = format(last_month, '%Y-%m')
    LastNMonths.append(last_month_)
    
LastNMonths = LastNMonths[::-1]
LastNMonths = LastNMonths[:-1]
LastNMonths

die letzten X Monate:  7


['2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07']